# Part 1 - Extracting and Saving Data from Yelp API

## Obective

- For this CodeAlong, we will be working with the Yelp API. 
- You will use the the Yelp API to search your home town for a cuisine type of your choice.
- Next class, we will then use Plotly Express to create a map with the Mapbox API to visualize the results.
    
    

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- Efficient API Calls Lesson Link: https://login.codingdojo.com/m/376/12529/88078

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
# os - for saving and loading files
# json - to work with json files
# math - to round up results
# time - to add a short pause to not overwhelm the server
import os, json, math, time

# to make yelpapi calls
from yelpapi import YelpAPI

# progress bar from tqdm_notebook
from tqdm.notebook import tqdm_notebook

In [2]:
!pip install yelpapi

## 1. Registering for Required APIs


- Yelp: https://www.yelp.com/developers/documentation/v3/get_started


> Check the official API documentation to know what arguments we can search for: https://www.yelp.com/developers/documentation/v3/business_search

### Load Credentials and Create Yelp API Object

In [3]:
# Load API Credentials
with open('/Users/austi/.secret/yelp_api.json', 'r') as f:
    login = json.load(f)

In [4]:
login.keys()

dict_keys(['client-id', 'api-key'])

In [5]:
# Instantiate YelpAPI Variable
yelp = YelpAPI(login['api-key'], timeout_s = 5.0)

### Define Search Terms and File Paths

In [6]:
# set our API call parameters and filename before the first call
location = "Cave Creek, AZ 85331"
term = "pizza"

location.split(',')[0]

'Cave Creek'

In [7]:
## Specify folder for saving data
FOLDER = 'Data/'

os.makedirs(FOLDER, exist_ok = True)
# Specifying JSON_FILE filename (can include a folder)
JSON_FILE = FOLDER+f"{location.split(',')[0]}-{term}.json"

### Check if Json File exists and Create it if it doesn't

In [8]:
JSON_FILE

'Data/Cave Creek-pizza.json'

In [9]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder, exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
    
    ## save the first page of results
    with open(JSON_FILE, 'w') as f:
        json.dump([], f)
        
## If it exists, inform user
else:
    print(f"[ii] {JSON_FILE} already exists.")

[i] Data/Cave Creek-pizza.json not found. Saving empty list to file.


### Load JSON FIle and account for previous results

### Make the first API call to get the first page of data

In [10]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp.search_query(term = term, location = location)

- We will use this first result to check:
    - how many total results there are?
    - Where is the actual data we want to save?
    - how many results do we get at a time?


In [11]:
## How many results total?
type(results)

dict

In [12]:
len(results)

3

In [13]:
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [14]:
results['total']

159

In [15]:
results['region']

{'center': {'longitude': -111.96922302246094, 'latitude': 33.8447133889471}}

- Where is the actual data we want to save?

In [16]:
results['businesses']

[{'id': 'yfXQDWscgk2TwUh82M51pQ',
  'alias': 'pizzicata-carefree-7',
  'name': 'Pizzicata',
  'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/IdtLhQ3ixlKsyHkwOReDzg/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/pizzicata-carefree-7?adjust_creative=_XtPN5sB9p2gFalQqwrzYQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=_XtPN5sB9p2gFalQqwrzYQ',
  'review_count': 352,
  'categories': [{'alias': 'pizza', 'title': 'Pizza'},
   {'alias': 'italian', 'title': 'Italian'},
   {'alias': 'cocktailbars', 'title': 'Cocktail Bars'}],
  'rating': 4.0,
  'coordinates': {'latitude': 33.825918, 'longitude': -111.92354},
  'transactions': ['delivery', 'pickup'],
  'price': '$$',
  'location': {'address1': '7212 E Ho Rd',
   'address2': None,
   'address3': '',
   'city': 'Carefree',
   'zip_code': '85377',
   'country': 'US',
   'state': 'AZ',
   'display_address': ['7212 E Ho Rd', 'Carefree, AZ 85377']},
  'phone': '+14804882848',
  'display_phone': '(480) 48

In [17]:
pd.DataFrame(results['businesses'])

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,yfXQDWscgk2TwUh82M51pQ,pizzicata-carefree-7,Pizzicata,https://s3-media1.fl.yelpcdn.com/bphoto/IdtLhQ...,False,https://www.yelp.com/biz/pizzicata-carefree-7?...,352,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 33.825918, 'longitude': -111.92354}","[delivery, pickup]",$$,"{'address1': '7212 E Ho Rd', 'address2': None,...",+14804882848,(480) 488-2848,4708.652245
1,91TLlpxUvRJcGlxedEw6YA,cowboy-pizza-cave-creek,Cowboy Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/ehpLQv...,False,https://www.yelp.com/biz/cowboy-pizza-cave-cre...,68,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 33.83065097618316, 'longitude': -...",[],NaN,"{'address1': '6920 E Cave Creek Rd', 'address2...",+14804882588,(480) 488-2588,3799.531370
2,cZxjPy4OrqJKscmCypDFoQ,pizzafarros-carefree-2,Pizzafarro's,https://s3-media2.fl.yelpcdn.com/bphoto/tsIfcP...,False,https://www.yelp.com/biz/pizzafarros-carefree-...,110,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",3.5,"{'latitude': 33.82213835670556, 'longitude': -...",[delivery],$$,"{'address1': '36889 N Tom Darlington Dr', 'add...",+14804880703,(480) 488-0703,4921.623074
3,IT0IODInyhKbG4a_ka71MQ,oreganos-cave-creek,Oregano's,https://s3-media2.fl.yelpcdn.com/bphoto/GTOjPf...,False,https://www.yelp.com/biz/oreganos-cave-creek?a...,462,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.5,"{'latitude': 33.82786616479023, 'longitude': -...",[delivery],$$,"{'address1': '37645 Cave Creek Rd', 'address2'...",+14808890386,(480) 889-0386,2302.624405
4,sZkmUH_oRUhCgb4uhoS-zQ,brugos-pizza-cave-creek-2,Brugo's Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/sPoZU2...,False,https://www.yelp.com/biz/brugos-pizza-cave-cre...,128,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.5,"{'latitude': 33.82982310051313, 'longitude': -...",[delivery],$$,"{'address1': '7100 E Cave Creek Rd', 'address2...",+14805753044,(480) 575-3044,4107.237085
5,__zA29wBG0LleSxMzNHpwQ,alberto-ristorante-italiano-cave-creek,Alberto Ristorante Italiano,https://s3-media1.fl.yelpcdn.com/bphoto/hUNtfu...,False,https://www.yelp.com/biz/alberto-ristorante-it...,149,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.0,"{'latitude': 33.827597, 'longitude': -111.924478}",[delivery],$$,"{'address1': '7171 E Cave Creek Rd', 'address2...",+14804885800,(480) 488-5800,4549.939478
6,Ijw0d-2wTcsciDesjCVgsA,barros-pizza-cave-creek-3,Barro's Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/gQBx4o...,False,https://www.yelp.com/biz/barros-pizza-cave-cre...,146,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.5,"{'latitude': 33.7654269550417, 'longitude': -1...",[delivery],$,"{'address1': '30815 N Cave Creek Rd', 'address...",+14804199161,(480) 419-9161,8918.737005
7,-EVBYBECSlt6FKE2ig4qAA,gios-pizza-anthem,Gio's Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/j0SHig...,False,https://www.yelp.com/biz/gios-pizza-anthem?adj...,271,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 33.870433, 'longitude': -112.150387}","[pickup, delivery]",$,"{'address1': '4205 W Anthem Way', 'address2': ...",+16235002350,(623) 500-2350,16966.434745
8,rnCkjgSojs5wqCaeyKoAVg,rays-pizza-cave-creek-phoenix,Ray's Pizza Cave Creek,https://s3-media3.fl.yelpcdn.com/bphoto/7UOWKJ...,False,https://www.yelp.com/biz/rays-pizza-cave-creek...,76,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 33.7979911, 'longitude': -111.980...",[delivery],$$,"{'address1': '4705 E Carefree Hwy', 'address2'...",+14804887823,(480) 488-7823,5295.704398
9,9ekvSQibalBglJH0ekuGlA,rosatis-pizza-cave-creek,Rosati's Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/8dhmF5...,False,https://www.yelp.com/biz/rosatis-pizza-cave-cr...,139,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.5,"{'latitude': 33.74249, 'longitude': -111.9775}","[pickup, delivery]",$$,"{'address1': '28325 N Tatum Blvd', 'addres

- Calculate how many pages of results needed to cover the total_results

In [18]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [19]:
(results['total'])/results_per_page

7.95

In [20]:
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total'])/results_per_page)
n_pages

8

In [21]:
for i in tqdm_notebook(range(1, n_pages+1)):
    ## The block of code we want to TRY to run
    try:

        time.sleep(.2)
        

        
        ## Read in results in progress file and check the length
        with open(JSON_FILE, 'r') as f:
            previous_results = json.load(f)
            
        ## save number of results for to use as offset
        n_results = len(previous_results)
        
        
        ## use n_results as the OFFSET 
        results = yelp.search_query(location = location, term = term, offset = n_results+1)

        ## append new results and save to file
        previous_results.extend(results['businesses'])

        with open(JSON_FILE, 'w') as f:
            json.dump(previous_results, f)

            
    ## What to do if we get an error/exception.
    except Exception as e:
        print(' [!] ERROR', e)

  0%|          | 0/8 [00:00<?, ?it/s]

## Open the Final JSON File with Pandas

In [22]:
df = pd.read_json(JSON_FILE)
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,91TLlpxUvRJcGlxedEw6YA,cowboy-pizza-cave-creek,Cowboy Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/ehpLQv...,False,https://www.yelp.com/biz/cowboy-pizza-cave-cre...,68,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 33.83065097618316, 'longitude': -...",[],"{'address1': '6920 E Cave Creek Rd', 'address2...",14804882588,(480) 488-2588,3799.531370,NaN
1,cZxjPy4OrqJKscmCypDFoQ,pizzafarros-carefree-2,Pizzafarro's,https://s3-media2.fl.yelpcdn.com/bphoto/tsIfcP...,False,https://www.yelp.com/biz/pizzafarros-carefree-...,110,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",3.5,"{'latitude': 33.82213835670556, 'longitude': -...",[delivery],"{'address1': '36889 N Tom Darlington Dr', 'add...",14804880703,(480) 488-0703,4921.623074,$$
2,IT0IODInyhKbG4a_ka71MQ,oreganos-cave-creek,Oregano's,https://s3-media2.fl.yelpcdn.com/bphoto/GTOjPf...,False,https://www.yelp.com/biz/oreganos-cave-creek?a...,462,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.5,"{'latitude': 33.82786616479023, 'longitude': -...",[delivery],"{'address1': '37645 Cave Creek Rd', 'address2'...",14808890386,(480) 889-0386,2302.624405,$$
3,sZkmUH_oRUhCgb4uhoS-zQ,brugos-pizza-cave-creek-2,Brugo's Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/sPoZU2...,False,https://www.yelp.com/biz/brugos-pizza-cave-cre...,128,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.5,"{'latitude': 33.82982310051313, 'longitude': -...",[delivery],"{'address1': '7100 E Cave Creek Rd', 'address2...",14805753044,(480) 575-3044,4107.237085,$$
4,__zA29wBG0LleSxMzNHpwQ,alberto-ristorante-italiano-cave-creek,Alberto Ristorante Italiano,https://s3-media1.fl.yelpcdn.com/bphoto/hUNtfu...,False,https://www.yelp.com/biz/alberto-ristorante-it...,149,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.0,"{'latitude': 33.827597, 'longitude': -111.924478}",[delivery],"{'address1': '7171 E Cave Creek Rd', 'address2...",14804885800,(480) 488-5800,4549.939478,$$


In [23]:
## convert the filename to a .csv.gz
csv_file = JSON_FILE.replace('.json','.csv.gz')
csv_file

'Data/Cave Creek-pizza.csv.gz'

In [24]:
## Save it as a compressed csv (to save space)
df.to_csv(csv_file, compression='gzip', index=False)

## Bonus: compare filesize with os module's `os.path.getsize`

In [25]:
size_json = os.path.getsize(JSON_FILE)
size_csv_gz = os.path.getsize(JSON_FILE.replace('.json','.csv.gz'))

print(f'JSON FILE: {size_json:,} Bytes')
print(f'CSV.GZ FILE: {size_csv_gz:,} Bytes')

print(f'the csv.gz is {size_json/size_csv_gz} times smaller!')

JSON FILE: 158,451 Bytes
CSV.GZ FILE: 23,079 Bytes
the csv.gz is 6.865592096711296 times smaller!


## Next Class: Processing the Results and Mapping 